In [36]:
import csv
import re

# The story file is expected to be tab delimited
fromFile = "storyville"

with open(f"{fromFile}.txt", "r", encoding="utf-8") as f:
    stories = csv.reader(f, delimiter="\t")
    with open(f"{fromFile}Stripped.txt", "w", encoding="utf-8") as newF:
        for storyID, story in enumerate(stories):
#             print(f"Story ID: {storyID}")
            storyNoIntro = story[1:]
            # Get rid of linespace
            storyNoIntro.pop(1)

            # get rid of actions and write to file
            for lineID, line in enumerate(storyNoIntro):
                lineStripped = re.sub("{.*}", "", line)
                lineStripped = re.sub("\[.*\]", "", lineStripped)
#                 print(lineID, lineStripped)
                newF.write(lineStripped+"\n")
            newF.write("<|endoftext|>\n")
#             print("<|endoftext|>")
